### Import Uber Movement Origin Distination Hourly Data

Uber origin destination zone data from the Movement project.  Each file contains hourly data, with one file for "am" times and another for "pm" times.<p>
    The first two columns of the data represent an origin zone and a destination zone.  These are followed by travel time statistics.<p>

In [1]:
import pandas as pd
from datetime import datetime

/home/mapdadmin/miniconda3/envs/flaxman2/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/mapdadmin/miniconda3/envs/flaxman2/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
!ls *.csv

20170101a.csv  20170101b.csv  20170210a_nocr.csv


In [3]:
date_in_filename = '20170101'
uber_csv = '{}a.csv'.format(date_in_filename)
df = pd.read_csv(uber_csv)

In [4]:
df.head()

sourceid  dstid  mov_year  mov_month  mov_day  mov_hod  tt_mean    tt_sd  \
0      2074   1860      2017          1        1        2  1314.00  1211.32   
1      2022   2042      2017          1        1        1    78.67    35.60   
2      2044   2092      2017          1        1        2   949.67  1807.45   
3      2081   1786      2017          1        1        2   615.33   399.86   
4      2054   2092      2017          1        1        1   108.38   134.80   

   tt_g_mean  tt_g_sd  
0     951.35     2.10  
1      73.16     1.42  
2     253.75     3.81  
3     533.74     1.62  
4      81.02     1.79

In [5]:
df.describe()

sourceid          dstid  mov_year  mov_month   mov_day  \
count  959217.000000  959217.000000  959217.0   959217.0  959217.0   
mean     4236.862684    4370.537264    2017.0        1.0       1.0   
std      1010.504381    1047.493005       0.0        0.0       0.0   
min       964.000000     911.000000    2017.0        1.0       1.0   
25%      3620.000000    3675.000000    2017.0        1.0       1.0   
50%      3909.000000    4077.000000    2017.0        1.0       1.0   
75%      4646.000000    5114.000000    2017.0        1.0       1.0   
max      7098.000000    7101.000000    2017.0        1.0       1.0   

             mov_hod        tt_mean          tt_sd      tt_g_mean  \
count  959217.000000  959217.000000  959217.000000  959217.000000   
mean        4.551861     542.675464     188.223773     508.225665   
std         3.527610     441.628853     181.833761     433.175388   
min         1.000000       2.400000       0.370000       2.030000   
25%         2.000000     232.860000      91.790000     201.300000   
50%         3.000000     427.600000     156.560000     392.970000   
75%         8.000000     716.720000     240.240000     680.480000   
max        11.000000    4953.670000    5022.180000    4686.930000   

             tt_g_sd  
count  959217.000000  
mean        1.527702  
std         0.503856  
min         1.000000  
25%         1.240000  
50%         1.390000  
75%         1.640000  
max        22.130000

In [6]:
# make a datetime field in default ISO format 
# which Omnisci will recognize
# 2018-11-02T17:45:51+00:00

In [7]:
df["ts"] = [datetime(mov_year, mov_month, mov_day, mov_hod) for mov_year, mov_month, mov_day, mov_hod in zip(df.mov_year, df.mov_month, df.mov_day, df.mov_hod)]

In [8]:
# must drop at least year since it is a reserved word
df.drop(columns=['mov_year', 'mov_month','mov_day','mov_hod'],inplace=True)

In [9]:
df.head()

sourceid  dstid  tt_mean    tt_sd  tt_g_mean  tt_g_sd                  ts
0      2074   1860  1314.00  1211.32     951.35     2.10 2017-01-01 02:00:00
1      2022   2042    78.67    35.60      73.16     1.42 2017-01-01 01:00:00
2      2044   2092   949.67  1807.45     253.75     3.81 2017-01-01 02:00:00
3      2081   1786   615.33   399.86     533.74     1.62 2017-01-01 02:00:00
4      2054   2092   108.38   134.80      81.02     1.79 2017-01-01 01:00:00

### Special case processing for the 'b' file

This represents pm, but data values already on 24 hour clock

In [18]:
uber_csv = '{}b.csv'.format(date_in_filename)
df_pm = pd.read_csv(uber_csv)

In [19]:
df_pm['mov_hod'].describe()

count    947957.000000
mean         17.302091
std           3.362363
min          12.000000
25%          14.000000
50%          17.000000
75%          20.000000
max          23.000000
Name: mov_hod, dtype: float64

In [20]:
df_pm["ts"] = [datetime(mov_year, mov_month, mov_day, mov_hod) for mov_year, mov_month, mov_day, mov_hod in zip(df_pm.mov_year, df_pm.mov_month, df_pm.mov_day, df_pm.mov_hod)]

In [21]:
df_pm.drop(columns=['mov_year', 'mov_month','mov_day','mov_hod'],inplace=True)

In [22]:
df_pm.head()

sourceid  dstid  tt_mean   tt_sd  tt_g_mean  tt_g_sd                  ts
0      5479   6069  1239.54  155.45    1230.71     1.12 2017-01-01 22:00:00
1      5339   5245   191.57   12.24     191.17     1.07 2017-01-01 17:00:00
2      5477   6002   809.38  251.22     776.96     1.32 2017-01-01 22:00:00
3      5391   5406   221.17  107.12     202.24     1.48 2017-01-01 20:00:00
4      5353   5758  1041.80  200.95    1026.04     1.18 2017-01-01 22:00:00

In [23]:
#ok, now hours cannot be conflated, so can just append am and pm tables
df = df.append(df_pm, sort=False)

In [24]:
df.head()

sourceid  dstid  tt_mean    tt_sd  tt_g_mean  tt_g_sd                  ts
0      2074   1860  1314.00  1211.32     951.35     2.10 2017-01-01 02:00:00
1      2022   2042    78.67    35.60      73.16     1.42 2017-01-01 01:00:00
2      2044   2092   949.67  1807.45     253.75     3.81 2017-01-01 02:00:00
3      2081   1786   615.33   399.86     533.74     1.62 2017-01-01 02:00:00
4      2054   2092   108.38   134.80      81.02     1.79 2017-01-01 01:00:00

### Set up for OmniSci data loading

In [25]:
# figure out where OmniSci log is, based on environment variables
import os
MAPD_STORAGE = %env MAPD_STORAGE
MAPD_ERROR_LOG = os.path.join(MAPD_STORAGE,'data/mapd_log/mapd_server.ERROR')

In [26]:
import sys,traceback
from pymapd import connect
con = connect(user="mapd", password="HyperInteractive!", host="localhost", dbname="mapd")

def mapdql(query):
    if con is None: # refresh if needed
        print('Connection bad or stale')
        return None
    try:
        print('Executing query: {}'.format(query))
        results = con.execute(query)
        return results
    except:
        print('Exception executing query')
        a,b,c = sys.exc_info()
        for d in traceback.format_exception(a,b,c) :
           print (d)
        return None

In [27]:
# drop any prior version to avoid confusion

table_name = 'uber_travel_times_{}'.format(date_in_filename)

query = 'DROP TABLE {};'.format(table_name)
try:
    print("Trying to drop prior table version if any")
    con.execute(query)
    print('...succeeded.')
except:
    print('Could not drop prior table version - perhaps nonexistent?')

Trying to drop prior table version if any
...succeeded.


In [28]:
query = 'CREATE TABLE {} ('.format(table_name)
query +=   'sourceid BIGINT ENCODING FIXED(16), '
query +=   'dstid BIGINT ENCODING FIXED(16), '
query +=   'tt_mean DOUBLE, tt_sd DOUBLE, tt_g_mean DOUBLE, tt_g_sd DOUBLE, '
query +=    'date_hour TIMESTAMP'
query += ');'
query

'CREATE TABLE uber_travel_times_20170101 (sourceid BIGINT ENCODING FIXED(16), dstid BIGINT ENCODING FIXED(16), tt_mean DOUBLE, tt_sd DOUBLE, tt_g_mean DOUBLE, tt_g_sd DOUBLE, date_hour TIMESTAMP);'

In [29]:
try:
    result = con.execute(query)
    if (result is None):
        print("Created table successfully.")
except:
    print('Exception creating table')
    a,b,c = sys.exc_info()
    for d in traceback.format_exception(a,b,c) :
       print (d)
    print('last line of Mapd Server Error log reads:')
    !tail -1 {MAPD_ERROR_LOG}

In [30]:
try:
    result = con.load_table(table_name, df) # , create=True
    if (result is None):
        print("Table loaded successfully.")
except:
    print('Exception executing query')
    a,b,c = sys.exc_info()
    for d in traceback.format_exception(a,b,c) :
       print (d)
    print('last line of Mapd Server Error log reads:')
    !tail -1 {MAPD_ERROR_LOG}

Table loaded successfully.


### Example of Geospatial Data Load from a geojson file zip archive

In [ ]:
# hmm, SF Open data TAZ polygons imported with string column types

In [ ]:
import_json_file = 'san_francisco_taz.json.zip'
geo_table = 'sf_taz_uber'

In [ ]:
import os
cwd_list = !pwd
cwd = cwd_list[0]
import_json_path = os.path.join(cwd, import_json_file)

In [ ]:
query = "COPY {} FROM '{}' WITH (GEO='True');".format(geo_table, import_json_path)

In [ ]:
try:
    result = con.execute(query) # , create=True
    if (result is None):
        print("geoJSON file loaded successfully.")
except:
    print('Exception executing query')
    a,b,c = sys.exc_info()
    for d in traceback.format_exception(a,b,c) :
       print (d)
    print('last line of Mapd Server Error log reads:')
    !tail -1 {MAPD_ERROR_LOG}

In [ ]:
query = 'ALTER TABLE {} ADD COLUMN {} BIGINT'.format(geo_table, 'taz_uber')

In [ ]:
try:
    result = con.execute(query) # , create=True
    if (result is None):
        print("alter table successfully.")
except:
    print('Exception executing query')
    a,b,c = sys.exc_info()
    for d in traceback.format_exception(a,b,c) :
       print (d)
    print('last line of Mapd Server Error log reads:')
    !tail -1 {MAPD_ERROR_LOG}

In [ ]:
query = 'UPDATE {} SET taz_uber CAST(movement_id as BIGINT)'.format(geo_table)